In [1]:
from pyEDM import *
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import fiona
import rasterio.mask
from descartes import PolygonPatch
from rasterio.plot import show
from tqdm import tqdm
import datetime
import random
from sklearn.preprocessing import StandardScaler
import scipy.stats
from datetime import datetime

In [306]:
observations_tables = '/home/rgreen/DroughtEDM/Data/observations_tables/'

In [ ]:
#ndvi_table = pd.read_csv(observations_tables + 'ndvi_table_anom.csv')

In [ ]:
#precip_table = pd.read_csv(obserations_tables + 'precip_table_anom.csv')

In [4]:
#temp_table = pd.read_csv(observations_tables + 'temp_table_anom.csv')

In [5]:
precip_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly/'
temp_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly/'
ndvi_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly/'

In [6]:
broadleaf_open_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_broadleaf_open/'
crop_irr_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_irr/'
crop_rain_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_rain/'
grassland_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_grassland/'
shrubland_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_shrubland/'

broadleaf_open_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_broadleaf_open_anom/'
crop_irr_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_irr_anom/'
crop_rain_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_rain_anom/'
grassland_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_grassland_anom/'
shrubland_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_shrubland_anom/'

In [7]:
def generate_dekads(start, end):
    '''
    Generates list of timestamps of dekads between two dates
    
    Parameters
    ----------
    start : pd.datetime object
        Start date of list
    end : pd.datetime object
        end date of list
    '''
    
    dtrange = pd.date_range(start, end)
    days = list(range(len(dtrange))) #length of dtrange
    daysDF = pd.DataFrame({'Days': days}, index=dtrange)
    d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 
    dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
    dekads = daysDF.groupby(dekaddates).mean()
    dekads = dekads.index
    
    return dekads

In [8]:
start = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)

dekads = generate_dekads(start, end)

In [9]:
def pixelwise_ts_table(in_dir, keyword):
    
    '''
    This function searches through the file directory and creates a dataframe of pixel values over time from rasters
    based on a given keyword describing the environmental variable of interest to generate a raster time series stack 
    
   In the output dataframe, columns are each pixel extracted from the raster (read squentially into a list)
   and rows are the same pixel over time (dekads going down)
    
    Args:
    - in_dir: path to the input directory
    -keyword: string that is unique to the environmental variable
        options: ['precip', 'temp', 'ndvi']
    '''
    
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains(keyword)
    files = files[tifs]
    #n_samples = len(files)
    
    
    pixelwise_TS = []
    
    for filename in tqdm(files): 
        
      
            open_file = xr.open_rasterio(in_dir+filename).sel(band=1)
            array = open_file.values

            pixel_list = array.ravel().tolist()
            
            pixelwise_TS.append(pixel_list)

    return pd.DataFrame(pixelwise_TS)

    

In [376]:
ndvi_table_anom

,0,1,2,3,4,5,6,7,8,9,...,392932,392933,392934,392935,392936,392937,392938,392939,392940,392941
0,1.101671,1.267218,0.938137,1.100588,1.324199,1.315054,1.165266,1.113739,0.995424,1.331126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.981070,1.093294,0.873698,1.124495,1.246212,1.232029,1.013628,0.881350,0.986223,1.310523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.620074,0.497290,0.323125,0.514614,1.014546,0.785064,0.773919,0.555548,0.556693,0.720380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.039260,0.837557,0.609005,0.773013,1.242413,1.024316,1.137690,0.987828,1.089278,1.286157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.971371,0.751863,0.467435,0.481555,1.118889,0.952852,1.045681,0.991793,0.930235,0.796915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,0.208073,0.026296,-0.284296,0.197938,-0.367827,-0.628079,-0.157488,-0.532811,-0.979176,-0.508893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,0.522922,0.110877,-0.002413,0.636307,-0.054164,-0.157637,0.066254,-0.048598,-0.132028,0.265398,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,0.331863,-0.081993,-0.378869,-0.353615,-0.445762,-0.467402,-0.187034,-0.058811,0.460797,0.651812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,0.306670,-0.136034,-0.296636,0.203851,-0.584455,-0.485288,-0.248363,-0.311625,-0.056771,0.484504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
precip_table_anom = pixelwise_ts_table(precip_anom_dir, 'precip')
temp_table_anom = pixelwise_ts_table(temp_anom_dir, 'temp')
ndvi_table_anom = pixelwise_ts_table(ndvi_anom_dir, 'ndvi')

100%|██████████| 606/606 [00:19<00:00, 30.47it/s]


In [11]:
precip_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'precip')
# temp_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'temp')
# ndvi_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir,  'ndvi')

100%|██████████| 606/606 [00:12<00:00, 50.03it/s]


In [12]:
precip_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'precip')
# temp_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'temp')
# ndvi_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir,  'ndvi')

100%|██████████| 606/606 [00:08<00:00, 68.17it/s]


In [13]:
precip_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'precip')
# temp_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'temp')
# ndvi_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir,  'ndvi')

100%|██████████| 606/606 [00:09<00:00, 65.15it/s]


In [14]:
precip_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'precip')
# temp_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'temp')
# ndvi_gr_table_anom = pixelwise_ts_table(grassland_anom_dir,  'ndvi')

100%|██████████| 606/606 [00:09<00:00, 65.48it/s]


In [15]:
precip_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'precip')
# temp_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'temp')
# ndvi_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir,  'ndvi')

100%|██████████| 606/606 [00:08<00:00, 67.72it/s]


In [285]:
bo_df = precip_bo_table_anom
cr_df = precip_cr_table_anom
ci_df = precip_ci_table_anom
gr_df = precip_gr_table_anom
sh_df = precip_sh_table_anom

df_list = [bo_df, cr_df, ci_df, gr_df, sh_df]
df_lct_list = []

for (df,lct) in zip(df_list,['woodland','cropland_rainfed','cropland_irrigated','grassland','shrubland']):
    
    df_t = df.T
    df_t['LC'] = np.nan
    df_t.LC[df_t.iloc[:,0].notna()] = lct
    
    df_lct_list.append(df_t.LC.dropna())
    
    #df_t.dropna()


In [290]:
#df_lct_list[0].LC.dropna().to_frame().merge(df_lct_list[1].LC.dropna().to_frame(),how='outer',left_index=True, right_index=True)
df_lct = pd.concat(df_lct_list)
df_lct.reindex(bo_df.T.index)
df_lct

16702      woodland
17870      woodland
22540      woodland
43545      woodland
43546      woodland
            ...    
392686    shrubland
392690    shrubland
392698    shrubland
392699    shrubland
392700    shrubland
Name: LC, Length: 137135, dtype: object

In [358]:
precip_table_anom_lct = precip_table_anom.T.iloc[df_lct.index, :]
temp_table_anom_lct = temp_table_anom.T.iloc[df_lct.index, :]
ndvi_table_anom_lct = ndvi_table_anom.T.iloc[df_lct.index, :]

In [363]:
precip_table_anom_lct.columns= dekads.strftime("%Y-%m-%d").tolist()
temp_table_anom_lct.columns= dekads.strftime("%Y-%m-%d").tolist()
ndvi_table_anom_lct.columns= dekads.strftime("%Y-%m-%d").tolist()

In [364]:
precip_table_anom_lct.insert(0, 'lc', df_lct)

In [365]:
temp_table_anom_lct.insert(0, 'lc', df_lct)

In [366]:
ndvi_table_anom_lct.insert(0, 'lc', df_lct)

In [367]:
precip_table_anom_lct.index.name='pixel_id'

In [368]:
temp_table_anom_lct.index.name='pixel_id'

In [369]:
ndvi_table_anom_lct.index.name='pixel_id'

In [374]:
precip_table_anom_lct[80000:84000]

,lc,2002-07-01,2002-07-11,2002-07-21,2002-08-01,2002-08-11,2002-08-21,2002-09-01,2002-09-11,2002-09-21,...,2019-01-21,2019-02-01,2019-02-11,2019-02-21,2019-03-01,2019-03-11,2019-03-21,2019-04-01,2019-04-11,2019-04-21
pixel_id,,,,,,,,,,,,,,,,,,,,,
252780,grassland,-0.487665,-0.487665,-0.487665,-0.487665,-0.487665,-0.487665,-0.487665,-0.487665,-0.387273,...,-0.314892,-0.379338,-0.341836,0.038343,-0.279637,-0.369789,0.177799,-0.383571,-0.406395,1.104448
252782,grassland,-0.456052,-0.456052,-0.456052,-0.456052,-0.456052,-0.456052,-0.456052,-0.456052,-0.380868,...,-0.286816,-0.354959,-0.317241,0.118993,-0.223350,-0.332648,0.172231,-0.333739,-0.394417,0.568577
252794,grassland,-0.493472,-0.493472,-0.493472,-0.493472,-0.493472,-0.474397,-0.433106,-0.393424,-0.425076,...,-0.408971,-0.432257,-0.397023,-0.309535,-0.377197,-0.428543,0.023775,-0.128067,-0.212884,1.374806
252795,grassland,-0.494614,-0.494614,-0.494614,-0.494614,-0.494614,-0.474623,-0.433915,-0.392054,-0.419644,...,-0.416105,-0.436621,-0.402424,-0.321402,-0.384258,-0.432737,-0.018818,-0.128486,-0.213081,1.520006
252797,grassland,-0.491822,-0.491822,-0.491822,-0.491822,-0.491822,-0.470432,-0.422915,-0.380508,-0.417212,...,-0.423220,-0.421808,-0.387971,-0.324433,-0.390820,-0.434248,0.008789,-0.164874,-0.208303,1.475049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91840,shrubland,0.301465,2.076340,3.003277,0.779261,1.741924,2.331781,1.245637,-0.037838,-0.225997,...,-0.711583,-0.719827,-0.719599,-0.717682,-0.716747,-0.715502,-0.703976,-0.672772,-0.635554,-0.634536
91842,shrubland,0.419315,1.923800,2.914677,0.868789,1.977928,2.260940,1.335006,-0.013727,-0.172701,...,-0.701566,-0.710834,-0.710558,-0.708868,-0.707791,-0.706995,-0.690114,-0.674163,-0.641941,-0.630054
91844,shrubland,0.502287,1.847087,2.873978,0.930792,1.925377,2.205865,1.327360,-0.050217,-0.157244,...,-0.689419,-0.697076,-0.696246,-0.696437,-0.698411,-0.698011,-0.681565,-0.659366,-0.621564,-0.624849


In [371]:
precip_table_anom_lct[80000:84000].to_csv(observations_tables + 'precip_table_anom_lct_80000_84000.csv')

In [372]:
temp_table_anom_lct[80000:84000].to_csv(observations_tables + 'temp_table_anom_lct_80000_84000.csv')

In [373]:
ndvi_table_anom_lct[80000:84000].to_csv(observations_tables + 'ndvi_table_anom_lct_80000_84000.csv')

In [238]:
# def create_lc_col(bo_df, cr_df, ci_df, gr_df, sh_df):
    
#     lc_col = pd.DataFrame()
    
#     for index,row in bo_df.iterrows():
#         if bo_df.loc[index,0]> -10 and bo_df.loc[index,0]< 10:
#             lc_col.loc[index,0]= 'woodland'
#         else:
#             #lc_col.loc[index,0]= np.nan
#             for index, row in cr_df.iterrows():
#                 if cr_df.loc[index,0]> -10 and cr_df.loc[index,0]< 10:
#                     lc_col.loc[index,0]= 'cropland_rainfed'
#                     #print('cropland_rainfed')
#                 else:            
#                     for index,row in ci_df.iterrows():
#                         if ci_df.loc[index,0]> -10 and ci_df.loc[index,0]< 10:
#                             lc_col.loc[index,0]= 'cropland_irrigated'
#                             #print('cropland_irrigated')
#                         else:
#                             for index,row in gr_df.iterrows():
#                                 if gr_df.loc[index,0]> -10 and gr_df.loc[index,0]< 10:
#                                     lc_col.loc[index,0]= 'grassland'
#                                    # print('grassland')
#                                 else:
#                                     for index,row in sh_df.iterrows():
#                                         if sh_df.loc[index,0]> -10 and sh_df.loc[index,0]< 10:
#                                             lc_col.loc[index,0]= 'shrubland'
#                                             #print('shrubland')
#                                         else:
#                                             lc_col.loc[index,0]= np.nan
#                                             #print(index)
   
#     return lc_col


In [ ]:
# def create_lc_col(bo_df, cr_df, ci_df, gr_df, sh_df):
    
#     #lc_col = pd.DataFrame()
#     lc_col = []
    
#     #for index,row in bo_df.iterrows():
#     for i in bo_df:
#         print(i)
#         if i> -10 and i< 10:
#        # if bo_df.loc[index,0]> -100 and bo_df.loc[index,0]< 100:
#             lc_col.append('woodland')
#             #lc_col.loc[i,0]= 'woodland'
#         else:
#             #lc_col.loc[index,0]= np.nan
#             #for index, row in cr_df.iterrows():
#             for i in cr_df:
#                 #if cr_df.loc[index,0]> -100 and cr_df.loc[index,0]< 100:
#                 if i> -10 and i< 10:
#                     #lc_col.loc[i,0]= 'cropland_rainfed'
#                     lc_col.append('cropland_rainfed')
#                     #print('cropland_rainfed')
#                 else:            
#                     for i in ci_df:
#                     #for index,row in ci_df.iterrows():
#                         #if ci_df.loc[index,0]> -100 and ci_df.loc[index,0]< 100:
#                         if i> -10 and i< 10:
#                             #lc_col.loc[i,0]= 'cropland_irrigated'
#                             lc_col.append('cropland_irrigated')
#                             #print('cropland_irrigated')
#                         else:
#                             for i in gr_df:
#                             #for index,row in gr_df.iterrows():
#                                 #if gr_df.loc[index,0]> -100 and gr_df.loc[index,0]< 100:
#                                 if i> -10 and i< 10:
#                                     #lc_col.loc[i,0]= 'grassland'
#                                     lc_col.append('grassland')
#                                    # print('grassland')
#                                 else:
#                                     for i in sh_df:
#                                     #for index,row in sh_df.iterrows():
#                                         #if sh_df.loc[index,0]> -100 and sh_df.loc[index,0]< 100:
#                                         if i> -10 and i< 10:
#                                             #lc_col.loc[i,0]= 'shrubland'
#                                             lc_col.append('shrubland')
#                                             #print('shrubland')
#                                         else:
#                                             #lc_col.loc[i,0]= np.nan
#                                             lc_col.append(np.nan)
#                                             #print(index)
   
#     return lc_col


In [ ]:
precip_lc_col_10 = create_lc_col(precip_bo_table_anom[0].T.head(10), precip_cr_table_anom[0].T.head(10), precip_ci_table_anom[0].T.head(10), precip_gr_table_anom[0].T.head(10), precip_sh_table_anom[0].T.head(10))



In [ ]:
precip_lc_col_10

In [ ]:
precip_lc_col = create_lc_col(precip_bo_table_anom.T, precip_cr_table_anom.T, precip_ci_table_anom.T, precip_gr_table_anom.T, precip_sh_table_anom.T)

In [117]:
len(precip_lc_col)

475558

In [85]:
precip_bo_table_anom.T.head(5)

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
precip_bo_table_anomT = precip_bo_table_anom.T

In [49]:
precip_bo_table_anom_0T= precip_bo_table_anom.T[0]

In [52]:
precip_bo_table_anom_0T

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
392937   NaN
392938   NaN
392939   NaN
392940   NaN
392941   NaN
Name: 0, Length: 392942, dtype: float64

In [60]:
np.isnan(np.nan)

True

In [64]:
precip_bo_table_anomT['lc'] = np.where(precip_bo_table_anomT[0]== np.isnan(np.nan), 'woodland', np.nan)

In [241]:
#precip_bo_table_anomT['lc'] = precip_bo_table_anomT[0].map({True: np.nan, False: "Hello"})

In [244]:
precip_bo_table_anomT

,0,1,2,3,4,5,6,7,8,9,...,597,598,599,600,601,602,603,604,605,lc
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
precip_bo_table_anomT.isnull.values.any()

AttributeError: 'function' object has no attribute 'values'

In [45]:
precip_bo_table_anom_0T[0]

nan

In [46]:
def fill_lc(df):
    if df == np.nan:
        return np.nan
    else:
        return 'woodland'
    

In [48]:
precip_bo_table_anom_0T['lc'] = precip_bo_table_anom_0T.apply(fill_lc, axis=0)

TypeError: fill_lc() got an unexpected keyword argument 'axis'

In [250]:
precip_bo_table_anom_trans_0 = precip_bo_table_anom.T[0]

In [251]:
precip_bo_table_anom_trans_0

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
392937   NaN
392938   NaN
392939   NaN
392940   NaN
392941   NaN
Name: 0, Length: 392942, dtype: float64

In [252]:
precip_bo_table_anom_trans_0['lc'] = np.where(precip_bo_table_anom_trans_0> -10, True, False)

In [253]:
precip_bo_table_anom_trans_0

0                                                       NaN
1                                                       NaN
2                                                       NaN
3                                                       NaN
4                                                       NaN
                                ...                        
392938                                                  NaN
392939                                                  NaN
392940                                                  NaN
392941                                                  NaN
lc        [False, False, False, False, False, False, Fal...
Name: 0, Length: 392943, dtype: object

In [ ]:
def create_lc_col(df):
    
    lc_col = pd.DataFrame()
    
    for i in df:
        if bo_df.loc[index,0]> -10 and bo_df.loc[index,0]< 10:
            lc_col.loc[index,0]= 'woodland'

In [271]:
def create_lc_col(bo_df):
    
    lc_col = pd.DataFrame()
    
    for index,row in bo_df.iterrows():
        if bo_df.loc[index,0]> -10 and bo_df.loc[index,0]< 10:
            lc_col.loc[index,0]= 'woodland'
        else:
            lc_col.loc[index,0] =np.nan
    return lc_col

In [273]:
create_lc_col(precip_bo_table_anom.T.head(100000))

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
99995,woodland
99996,woodland
99997,woodland
99998,NaN


In [259]:
precip_bo_table_anom.T.head(10)

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
